### Reading data and template

To get started, read in the data and template files, which can both be downloaded using the links in the yellow box above. Both the data and the template have a sampling rate of 4096 Hz. 

The data is 16 seconds of simulated, colored noise with a signal added somewhere in the first 12 seconds. The template is a 1 second inspiral-like signal. The template has been tapered at the beginning and end, so that it can be more easliy Fourier transformed.


In [1]:
# Get python libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import h5py

In [3]:
# Read the data file
fs         = 4096 # Sampling at 4096 Hz
dfile      = h5py.File('data_w_signal.hdf5', 'r')
data      = dfile['strain/Strain'][...]
dfile.close()
time       = np.arange(0, 16, 1./fs)

In [ ]:
# Now, we read the template file
tempfile = h5py.File('template.hdf5', 'r')
template = tempfile['strain/strain'][...]
temp_time = np.arange